In [0]:
#Ingest file to volumne

file_name = "2024-wimbledon-points.csv"
source_url = f"https://raw.githubusercontent.com/JeffSackmann/tennis_slam_pointbypoint/master/{file_name}"
volume_path = "/Volumes/workspace/default/my_data_volume"
full_path = f"{volume_path}/{file_name}"

# Create the volume if it doesn't exist
spark.sql(
    "CREATE VOLUME IF NOT EXISTS workspace.default.my_data_volume"
)

# Copy the file from the URL to the volume
dbutils.fs.cp(
    source_url,
    full_path
)




In [0]:
#Bronze ingestion
# Read the file from the volume
df_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(full_path)
)

display(df_raw)

#Create Unity Catalog Table
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.bronze")

(
    df_raw.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.bronze.raw_wimbledon_points_2024")
)